<a href="https://colab.research.google.com/github/ZacariDigital/Generative-AI-Absract-Art/blob/main/Generative_A_I_Abstract_ART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/openai/CLIP
!git clone https://github.com/CompVis/taming-transformers

Cloning into 'CLIP'...
remote: Enumerating objects: 243, done.
remote: Total 243 (delta 0), reused 0 (delta 0), pack-reused 243
Receiving objects: 100% (243/243), 8.92 MiB | 2.29 MiB/s, done.
Resolving deltas: 100% (123/123), done.
Cloning into 'taming-transformers'...
remote: Enumerating objects: 1339, done.


In [ ]:
##Installing extra libraries
!pip install --no-deps ftfy regex tqdm
!pip install omegaconf==2.0.0 pytorch-lightning==1.0.8
!pip uninstall torchtext --yes
!pip install einops
!pip install torchvision==1.13.1

In [ ]:
!pip install torch==1.13.1



In [ ]:
##importing libaries
import numpy as np
import torch, os, imageio, pdb, math
import torchvision 
import torchvision.transforms as T
import torchvision.transforms.functional as TF

import PIL 
import matplotlib.pyplot as plt 

import yaml 
from omegaconf import OmegaConf

from CLIP import clip
#import warnings
#warnings.filterwarnings('ignore')

In [ ]:
##Coding Helper functions


def show_from_tensor(tensor):
  img = tensor.clone()
  img = img.mul(255).byte()
  img = img.cpu().numpy().transpose(1,2,0)

  plt.figure(figsize=(10,7))
  plt.axis('off')
  plt.imshow(img)
  plt.show()


def norm_data(data):
  return(data.clip(-1,1)+1)/2 ##range between 0 & 1 in the result

##Parameters
learning_rate = .5
batch_size = 1
wd = .1
noise_factor = .1


total_iter=100
im_shape = [225, 400, 3] #h,w,channel
size1, size2, channels = im_shape


In [ ]:
## CLIP MODEL ##
clipmodel, _ = clip.load('ViT-B/32', jit=False)
clipmodel.eval()
print(clip.available_models())

print("Clip model visual input resolution", clipmodel.visual.input_resolution)
device=torch.device("cuda:0")
torch.cuda.empty_cache

In [ ]:
##Taming Transformer##

%cd /content/taming-transformers

!mkdir -p models/vqgan_imagenet_f16_16384/checkpoints
!mkdir -p models/vqgan_imagenet_f16_16384/configs

if len(os.listdir('models/vqgan_imagenet_f16_16384/checkpoints/')) == 0:
  !wget 'https://heibox.uni-heidelberg.de/f/867b05fc8c4841768640/?dl=1' -O 'models/vqgan_imagenet_f16_16384/checkpoints/last.ckpt'
  !wget 'https://heibox.uni-heidelberg.de/f/274fb24ed38341bfa753/?dl=1' -O 'models/vqgan_imagenet_f16_16384/configs/model.yaml'


In [ ]:
from taming.models.vqgan import VQModel

def load_config(config_path, display=False):
  config_data = OmegaConf.load(config_path)
  if display:
    print(yaml.dump(OmegaConf.to_container(config_data)))
  return config_data

def load_vqgan(config, chk_path=None):
  model = VQModel(**config.model.params)
  if chk_path is not None:
    state_dict = torch.load(chk_path, map_location="cpu")["state_dict"]
    missing, unexpected = model.load_state_dict(state_dict, strict=False)
  return model.eval()

def generator(x):
  x = taming_model.post_quant_conv(x)
  x = taming_model.decoder(x)
  return x

taming_config = load_config("./models/vqgan_imagenet_f16_16384/configs/model.yaml", display=True)
taming_model = load_vqgan(taming_config, chk_path="./models/vqgan_imagenet_f16_16384/checkpoints/last.ckpt").to(device)



In [ ]:

##Declare the values that we are going to optimize##

class Parameters(torch.nn.Module):
  def __init__(self):
    super(Parameters, self).__init__()
    self.data = .5*torch.randn(batch_size, 256, size1//16, size2//16).cuda() #1x256x14x25 (225/16, 400/16)
    self.data = torch.nn.Parameter(torch.sin(self.data))

  def forward(self):
    return self.data

def init_params():
  params=Parameters().cuda()
  optimizer = torch.optim.Adam([{'params':[params.data], 'lr': learning_rate}], weight_decay=wd)
  return params, optimizer

In [ ]:
## Encoding Prompts 

normalize = torchvision.transforms.Normalize((0.48145466, 0.4578275, 0.408210732),(0.26862954, 0.26130258, 0.27577711))

def encodeText(text):
  t=clip.tokenize(text).cuda()
  t=clipmodel.encode_text(t).detach().clone()

def createEncodings(include, exclude, extras):
  include_enc=[]
  for text in include:
    include_enc.append(encodeText(text))
  exclude_enc=encodeText(exclude) if exclude !='' else 0
  extras_enc=encodeText(extras) if extras !='' else 0

  return include_enc, exclude_enc, extras_enc

augTransform = torch.nn.Sequential(
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomAffine(30, (.2,.2), fill=0)

).cuda()

Params, optimizer = init_params()

with torch.no_grad():
  print(Params().shape)
  img = norm_data(generator(Params()).cpu())
  print("img dimension", img.shape)
  show_from_tensor(img[0])


In [ ]:
## crops

def create_crops(img, num_crops=30):
  p=size1/2
  img = torch.nn.functional.pad(img, (p,p,p,p), mode='constant', value=0)

  img = augTransforms(img)

  crop_set = []
  for ch in range(num_crops):
    gap1=int(torch.normal(1.0, .5, ()).clip(.2, 1.5) * size1)
    gap2=int(torch.normal(1.0, .5, ()).clip(.2, 1.5) * size1)
    offsetx = torch.randint(0, int(size1*2-gap1),())
    offsety = torch.randint(0, int(size1*2-gap1),())

    crop=img[:,:,offsetx:offsetx+gap2, offsety:offsety+gap2]

    crop = torch.nn.fuctional.interpolate(crop,(224,224), mode='blinear', align_corners=True)
    crop_set.append(crop)

  img_crops = torch.cat(crop_set,0)

  img_crops = img_crops + noise_factor*torch.radn_like(img_crops, requires_grad=False)

  return img_crops




In [ ]:
## show current state of generation

def showme(Params, show_crop):
  with torch.no_grad():
    generated = generator(Params())

    if (show_crop):
      print("Augmented cropped example")
      aug_gen = generated.float()
      aug_gen = create_crops(aug_gen, num_crops=1)
      aug_gen_norm = norm_data(aug_gen[0])
      show_from_tensor(aug_gen_norm)

      print("Generation")
      latest_gen=norm_data(generated.cpu())
      show_from_tensor(latest_gen[0])

    return (latest_gen[0])


In [ ]:
## Optimization Process


def optimize_result(Params, prompt):
  alpha=1 ##importance to include encodings
  beta=.5  ##importance to exclude encodings

  out = generator(Params())
  out = norm_data(out)
  out = create_crops(out)
  out = normalize(out)
  image_enc=clipmodel.encode_image(out)

  ## text encoding w1 & w2
  final_enc = w1*prompt + w2*extras_enc
  final_text_include_enc = final_enc / final_enc.norm(dim=-1, keepdim=True)
  final_text_exclude_enc = exclude_enc

  ##calculating the loss
  main_loss = torch.cosine_similarity(final_text_include_enc, img_enc, -1)
  penalize_loss = torch.cosine.similarity(final_text_exclude_enc, img_enc, -1)

  final_loss = -alpha*main_loss + beta*penalize_loss

def optimize(Params, optimizer, prompt):
  loss = optimize_result(Params, prompt).mean()
  optimizer.zero_grad()
  loss.backwards()
  optimizer.step()
  return loss





In [ ]:
## training loop

def training_loop(Params, optimizer, show_crop=False):
  res_img=[]
  res_z=[]

  for prompt in include_enc:
    iteration=0
    Params, optimizer = init_params()

    for it in range(total_iter):
      loss = optimize(Params, optimizer, prompt)

      if iteration>0 and iteration%(total_iter-1)==0:
        new_img = showme(Params, show_crop)
        res_img.append(new_img)
        res_z.append(Params())
        print("loss", loss.items(), "\iteration:", iteration)

      
      iteration+=1
    torch.cuda.empty_cache()
  return res_img, res_z

